# Diversification

In [1]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from cvxopt import matrix
from cvxopt.solvers import qp as Solver, options as SolverOptions

### Inputs

In [2]:
##### Inputs
# Expected returns
MNS = np.array([0.06, 0.065, 0.08])

# Standard deviations
SDS = np.array([0.15, 0.165, 0.21])

# Correlations
C  = np.identity(3)
C[0, 1] = C[1, 0] = 0.75
C[0, 2] = C[2, 0] = 0.75
C[1, 2] = C[2, 1] = 0.75

# Covariance matrix
COV = np.diag(SDS) @ C @ np.diag(SDS)

print(COV)

[[0.0225    0.0185625 0.023625 ]
 [0.0185625 0.027225  0.0259875]
 [0.023625  0.0259875 0.0441   ]]


In [3]:
# Portfolio weights
WGTS = np.array([0.25, 0.5, 0.25])

#Portfolio expected return
port_expret = WGTS @ MNS
print(f'Portfolio Expected Return:\t {port_expret: ,.4f}')

# Portfolio standard deviation
port_sd = np.sqrt(WGTS @ COV @ WGTS)
print(f'Portfolio Standard Deviation:\t {port_sd: ,.4f}')

Portfolio Expected Return:	  0.0675
Portfolio Standard Deviation:	  0.1583


### Investment opportunity set
First, let's consider the set of just investing in asset 1 and asset 2

In [4]:
WGT1 = np.arange(0,1.05,0.05)
df = pd.DataFrame(dtype='float',columns = ['w1','w2','w3','port_expret','port_sd'], index=np.arange(len(WGT1)))
df.w1 = WGT1
df.w2 = 1-WGT1
df.w3 = 0
df.port_expret = [[w,1-w,0] @ MNS for w in WGT1]
df.port_sd = [np.sqrt([w,1-w,0] @ COV @ [w,1-w,0]) for w in WGT1]

In [5]:
fig = go.Figure()

# Plot portfolios
trace= go.Scatter(x=df.port_sd, y=df.port_expret, name='Portfolios')
fig.add_trace(trace)

# Plot underlying assets
trace1= go.Scatter(x=[SDS[0]], y=[MNS[0]],mode="markers", marker=dict(size=10, color="red"), name='Asset 1')
fig.add_trace(trace1)
trace2= go.Scatter(x=[SDS[1]], y=[MNS[1]],mode="markers", marker=dict(size=10, color="red"), name='Asset 2')
fig.add_trace(trace2)

# Formatting
fig.layout.yaxis["title"] = "Expected Return"
fig.layout.xaxis["title"] = "Standard Deviation"
fig.update_yaxes(tickformat=".1%")
fig.update_xaxes(tickformat=".1%")
fig.update_xaxes(range=[0.85 * df["port_sd"].min(), 1.25 * df["port_sd"].max()])
fig.update_yaxes(range=[0.85 * df["port_expret"].min(), 1.25 * df["port_expret"].max()])
fig.update_layout(legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01))
fig.show()

Let's add some of asset 3 now.  We will simulate weights that are non-negative and sum to 1.


In [6]:
def simulate_weights(n):
    ''' Simulate non-negative weights that sum to 1.'''
    x = np.random.exponential(scale=1.0, size=n)
    return x / sum(x)
simulate_weights(3)
# simulate_weights(3).sum()

array([0.30804601, 0.47178612, 0.22016787])

In [7]:
num_sims = 500
df = pd.DataFrame(dtype='float',columns = ['w1','w2','w3','port_expret','port_sd'], index=np.arange(num_sims))
for i in df.index:
    wgts = simulate_weights(3)
    df.loc[i,['w1','w2','w3']] = wgts
    df.loc[i,'port_expret'] = wgts @ MNS
    df.loc[i,'port_sd'] = np.sqrt(wgts @ COV @ wgts)
df.head()

,w1,w2,w3,port_expret,port_sd
0,0.549491,0.330444,0.120065,0.064054,0.149407
1,0.317636,0.145703,0.536661,0.071462,0.171423
2,0.645725,0.113278,0.240997,0.065386,0.154207
3,0.016917,0.766479,0.216604,0.068164,0.165389
4,0.344663,0.536064,0.119272,0.065066,0.152475


In [8]:
fig = go.Figure()

# Plot portfolios
trace= go.Scatter(x=df.port_sd, y=df.port_expret,mode="markers", name='Portfolios')
fig.add_trace(trace)
fig.layout.yaxis["title"] = "Expected Return"
fig.layout.xaxis["title"] = "Standard Deviation"

# Plot underlying assets
trace1= go.Scatter(x=[SDS[0]], y=[MNS[0]],mode="markers", marker=dict(size=10, color="red"), name='Asset 1')
fig.add_trace(trace1)
trace2= go.Scatter(x=[SDS[1]], y=[MNS[1]],mode="markers", marker=dict(size=10, color="red"), name='Asset 2')
fig.add_trace(trace2)
trace3= go.Scatter(x=[SDS[2]], y=[MNS[2]],mode="markers", marker=dict(size=10, color="red"), name='Asset 3')
fig.add_trace(trace3)

# Formatting
fig.update_yaxes(tickformat=".1%")
fig.update_xaxes(tickformat=".1%")
fig.update_xaxes(range=[0.7 * df["port_sd"].min(), 1.25 * df["port_sd"].max()])
fig.update_yaxes(range=[0.7 * df["port_expret"].min(), 1.25 * df["port_expret"].max()])
fig.update_layout(legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01))
fig.show()

### Allowing short sales

In [9]:
def simulate_weights(n):
    ''' Simulate weights that sum to 1.'''
    x = np.random.uniform(low=-0.5, high=2, size=n-1)
    x = np.append(x, 1-x.sum())
    return x 
simulate_weights(3)
# simulate_weights(3).sum()

array([ 0.69660837, -0.13275239,  0.43614402])

In [10]:
num_sims = 500
df = pd.DataFrame(dtype='float',columns = ['w1','w2','w3','port_expret','port_sd'], index=np.arange(num_sims))
for i in df.index:
    wgts = simulate_weights(3)
    df.loc[i,['w1','w2','w3']] = wgts
    df.loc[i,'port_expret'] = wgts @ MNS
    df.loc[i,'port_sd'] = np.sqrt(wgts @ COV @ wgts)
df.head()

,w1,w2,w3,port_expret,port_sd
0,-0.036979,1.665248,-0.628268,0.055761,0.193309
1,1.192450,0.661827,-0.854277,0.046224,0.166981
2,0.239937,1.736158,-0.976095,0.049159,0.204192
3,0.245192,0.290382,0.464426,0.070740,0.167909
4,1.173444,0.972220,-1.145664,0.041948,0.188518


In [11]:
fig = go.Figure()
# Plot random portfolios
string =  "Asset 1: %{customdata[0]:.1%}<br>"
string += "Asset 2: %{customdata[1]:.1%}<br>"
string += "Asset 3: %{customdata[2]:.1%}<br>"
string += "<extra></extra>"
trace= go.Scatter(x=df.port_sd, y=df.port_expret,mode="markers",
    customdata=df[['w1','w2','w3']],hovertemplate=string, name='Random Portfolios')
fig.add_trace(trace)

# Plot underlying assets
trace1= go.Scatter(x=[SDS[0]], y=[MNS[0]],mode="markers", marker=dict(size=10, color="red"), name='Asset 1')
fig.add_trace(trace1)
trace2= go.Scatter(x=[SDS[1]], y=[MNS[1]],mode="markers", marker=dict(size=10, color="red"), name='Asset 2')
fig.add_trace(trace2)
trace3= go.Scatter(x=[SDS[2]], y=[MNS[2]],mode="markers", marker=dict(size=10, color="red"), name='Asset 3')
fig.add_trace(trace3)

# Formatting
fig.layout.yaxis["title"] = "Expected Return"
fig.layout.xaxis["title"] = "Standard Deviation"
fig.update_yaxes(tickformat=".1%")
fig.update_xaxes(tickformat=".1%")
fig.update_xaxes(range=[0.7 * df["port_sd"].min(), 1.25 * df["port_sd"].max()])
fig.update_yaxes(range=[0.7 * df["port_expret"].min(), 1.25 * df["port_expret"].max()])
fig.update_layout(legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01))
fig.show()

### Efficient portfolios

The function `cvxopt.solvers.qp` solves problems of the general form:
\begin{align*}
    \underset{w}{\text{min  }}& \frac{1}{2} w' Q w + p'w \\
     \text{subject to  } & Gw \le h \\
                        & Aw = b \\
\end{align*}

In [12]:
def frontier(means, cov, target):
    n = len(means)
    Q = matrix(cov, tc="d")
    p = matrix(np.zeros(n), (n, 1), tc="d")
    # Constraint: short-sales allowed
    G = matrix(np.zeros((n,n)), tc="d")
    h = matrix(np.zeros(n), (n, 1), tc="d")
    # Fully-invested constraint + E[r] = target
    A = matrix(np.vstack((np.ones(n), means)), (2, n), tc="d")
    b = matrix([1, target], (2, 1), tc="d")
    sol = Solver(Q, p, G, h, A, b)
    wgts = np.array(sol["x"]).flatten() if sol["status"] == "optimal" else np.array(n * [np.nan])
    return wgts
SolverOptions['show_progress'] = False

TARGET_EXP_RET = 0.07
frontier(MNS, COV, TARGET_EXP_RET)

array([0.25461741, 0.32717678, 0.4182058 ])

We can trace out the frontier by solving for the minimum variance portfolio for a number of expected return targets.

In [13]:
NUM_TARGETS = 20
df = pd.DataFrame(dtype='float',columns=['target_expret','w1','w2','w3','port_expret','port_sd'], index=np.arange(NUM_TARGETS))
df.target_expret = np.linspace(0.05, 0.10,NUM_TARGETS)
for i in df.index:
    wgts = frontier(MNS, COV, df.loc[i,'target_expret'])
    df.loc[i,['w1','w2','w3']] = wgts
    df.loc[i,'port_expret'] = wgts @ MNS
    df.loc[i,'port_sd'] = np.sqrt(wgts @ COV @ wgts)
df.head()

,target_expret,w1,w2,w3,port_expret,port_sd
0,0.050000,1.151715,0.464380,-0.616095,0.050000,0.154641
1,0.052632,1.033676,0.446327,-0.480003,0.052632,0.148910
2,0.055263,0.915637,0.428274,-0.343911,0.055263,0.145249
3,0.057895,0.797598,0.410221,-0.207818,0.057895,0.143819
4,0.060526,0.679558,0.392168,-0.071726,0.060526,0.144683


In [14]:
fig = go.Figure()
# Plot frontier
string =  "Asset 1: %{customdata[0]:.1%}<br>"
string += "Asset 2: %{customdata[1]:.1%}<br>"
string += "Asset 3: %{customdata[2]:.1%}<br>"
string += "<extra></extra>"
trace= go.Scatter(x=df.port_sd, y=df.port_expret,mode="lines",
    customdata=df[['w1','w2','w3']],hovertemplate=string, name='Frontier')
fig.add_trace(trace)

# Plot underlying assets
trace1= go.Scatter(x=[SDS[0]], y=[MNS[0]],mode="markers", marker=dict(size=10, color="red"), name='Asset 1')
fig.add_trace(trace1)
trace2= go.Scatter(x=[SDS[1]], y=[MNS[1]],mode="markers", marker=dict(size=10, color="red"), name='Asset 2')
fig.add_trace(trace2)
trace3= go.Scatter(x=[SDS[2]], y=[MNS[2]],mode="markers", marker=dict(size=10, color="red"), name='Asset 3')
fig.add_trace(trace3)

# Formatting
fig.layout.yaxis["title"] = "Expected Return"
fig.layout.xaxis["title"] = "Standard Deviation"
fig.update_yaxes(tickformat=".1%")
fig.update_xaxes(tickformat=".1%")
fig.update_xaxes(range=[0.7 * df["port_sd"].min(), 1.25 * df["port_sd"].max()])
fig.update_yaxes(range=[0.7 * df["port_expret"].min(), 1.25 * df["port_expret"].max()])
fig.update_layout(legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01))
fig.show()

### Global minimum variance

Note there is no longer a target expected return constraint

In [15]:
import numpy as np
from cvxopt import matrix
from cvxopt.solvers import qp as Solver, options as SolverOptions

def gmv(means, cov):
    n = len(means)
    Q = matrix(cov, tc="d")
    p = matrix(np.zeros(n), (n, 1), tc="d")
    # Constraint: short-sales allowed
    G = matrix(np.zeros((n,n)), tc="d")
    h = matrix(np.zeros(n), (n, 1), tc="d")
    # Constraint: fully-invested portfolio
    A = matrix(np.ones(n), (1, n), tc="d")
    b = matrix([1], (1, 1), tc="d")
    sol = Solver(Q, p, G, h, A, b)
    wgts = np.array(sol["x"]).flatten() if sol["status"] == "optimal" else np.array(n * [np.nan])
    return wgts
SolverOptions['show_progress'] = False

wgts_gmv = gmv(MNS, COV)
wgts_gmv

array([ 0.78298611,  0.40798611, -0.19097222])

In [16]:
fig = go.Figure()

# Plot frontier
string =  "Asset 1: %{customdata[0]:.1%}<br>"
string += "Asset 2: %{customdata[1]:.1%}<br>"
string += "Asset 3: %{customdata[2]:.1%}<br>"
string += "<extra></extra>"
trace= go.Scatter(x=df.port_sd, y=df.port_expret,mode="lines",
    customdata=df[['w1','w2','w3']],hovertemplate=string, name='Frontier')
fig.add_trace(trace)

# Plot underlying assets
trace1= go.Scatter(x=[SDS[0]], y=[MNS[0]],mode="markers", marker=dict(size=10, color="red"), name='Asset 1')
fig.add_trace(trace1)
trace2= go.Scatter(x=[SDS[1]], y=[MNS[1]],mode="markers", marker=dict(size=10, color="red"), name='Asset 2')
fig.add_trace(trace2)
trace3= go.Scatter(x=[SDS[2]], y=[MNS[2]],mode="markers", marker=dict(size=10, color="red"), name='Asset 3')
fig.add_trace(trace3)

# Plot GMV
trace= go.Scatter(x=[np.sqrt(wgts_gmv @ COV @ wgts_gmv)], y=[wgts_gmv @ MNS] ,
    mode="markers", marker=dict(size=10, color="black"),
    customdata=wgts_gmv.reshape(1,3),hovertemplate=string, name='GMV')
fig.add_trace(trace)

# Formatting
fig.layout.yaxis["title"] = "Expected Return"
fig.layout.xaxis["title"] = "Standard Deviation"
fig.update_yaxes(tickformat=".1%")
fig.update_xaxes(tickformat=".1%")
fig.update_xaxes(range=[0.7 * df["port_sd"].min(), 1.25 * df["port_sd"].max()])
fig.update_yaxes(range=[0.7 * df["port_expret"].min(), 1.25 * df["port_expret"].max()])
fig.update_layout(legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01))
fig.show()